In [32]:
import nltk
import random
import os
import json
from nltk.stem import  PorterStemmer
import random
import pickle
import numpy as np
from collections import Counter
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn

In [2]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset_new'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,each_file) for sentence in data_temp]
    return dataset

In [4]:
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)

In [7]:
def tags(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [94]:
def get_polarity(word,tag):
    #print 'Get Polarity'
    #print 'Word : '+str(word)+' Tag : '+str(tag)
    synset = swn.senti_synsets(word,tag)
    if(len(synset)>0):
        pos_pol=synset[0].pos_score()
        neg_pol=synset[0].neg_score()
        #print str(pos_pol)+' '+str(neg_pol)
        #print ' '    
        if(pos_pol == neg_pol):
            return 0
        if(pos_pol>neg_pol):
            return pos_pol
        else:
            return (-neg_pol)
    return 0   

In [72]:
def frange(start, end, step):
    tmp = start
    while(tmp < end):
        yield tmp
        tmp += step  

In [89]:
def extract_features(sentence):       
    features =Counter()
    for i in frange(-1,1,0.1):
        features[round(i,1)]=0
    pos=nltk.pos_tag(sentence.split())
    for word,tag in pos:
        tag=tags(tag)
        polarity=get_polarity(word,tag)
        features[round(polarity,1)] += 1
    feature_vector=[]
    for i in frange(-1,1,0.1):
        feature_vector.append(features[round(i,1)])    
    return feature_vector

In [90]:
extract_features('this is a very good company')

Get Polarity
Word : this Tag : None
Get Polarity
Word : is Tag : v
0.25 0.125
 
Get Polarity
Word : a Tag : None
0.0 0.0
 
Get Polarity
Word : very Tag : r
0.25 0.25
 
Get Polarity
Word : good Tag : a
0.75 0.0
 
Get Polarity
Word : company Tag : n
0.0 0.0
 


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]

In [91]:
train_test = nltk.classify.apply_features(extract_features, labeled_data)

In [92]:
#dividing into training and test data
train_set=train_test[:-10000]
test_set=train_test[-10000:]

In [98]:
print labeled_data[1]
train_set[1]

(u'position outsourced', 'neutral')


([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'neutral')

In [95]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
with open('senti_naive_bayes_unigram_model','w') as f:
    pickle.dump(classifier,f)

AttributeError: 'list' object has no attribute 'items'